In [ ]:
# 🚀 UAT STAGING PROMOTION - ALWAYS BEST MODEL LOGIC

import mlflow
from mlflow.tracking import MlflowClient
import time
import yaml
import sys
import traceback
from typing import Optional, Dict, List
from datetime import datetime

print("=" * 80)
print("🚀 UAT STAGING PROMOTION - ALWAYS BEST MODEL")
print("=" * 80)

# =============================================================================
# ✅ LOAD PIPELINE CONFIGURATION
# =============================================================================
print("\n📋 Step 1: Loading configuration from pipeline_config.yml...")

try:
    import os
    repo_path = os.getcwd()
    config_path = os.path.join(repo_path, "pipeline_config.yml")

    with open(config_path, "r") as f:
        pipeline_cfg = yaml.safe_load(f)
    
    print(f"✅ Configuration loaded successfully from → {config_path}")

except FileNotFoundError:
    print(f"❌ ERROR: pipeline_config.yml not found in {repo_path}")
    sys.exit(1)
except Exception as e:
    print(f"❌ ERROR loading configuration: {e}")
    traceback.print_exc()
    sys.exit(1)


# =============================================================================
# ✅ CONFIGURATION CLASS
# =============================================================================
class Config:
    """Configuration manager - reads from pipeline_config.yml"""
    
    def __init__(self):
        MODEL_TYPE = pipeline_cfg["model"]["type"]
        UC_CATALOG = pipeline_cfg["model"]["catalog"]
        UC_SCHEMA = pipeline_cfg["model"]["schema"]
        BASE_NAME = pipeline_cfg["model"]["base_name"]
        
        self.MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.{BASE_NAME}_{MODEL_TYPE}"
        self.MODEL_TYPE = MODEL_TYPE

        self.STAGING_ALIAS = pipeline_cfg["aliases"]["staging"]
        self.PRODUCTION_ALIAS = pipeline_cfg["aliases"]["production"]
        self.BEST_ALIAS = pipeline_cfg["aliases"]["best"]
        
        self.PRIMARY_METRIC = pipeline_cfg["metrics"]["classification"]["primary_metric"]
        self.DIRECTION = pipeline_cfg["metrics"]["classification"]["direction"]
        self.TOLERANCE = 1e-6
        
        print(f"\n📊 Configuration Summary:")
        print(f"   Model Name: {self.MODEL_NAME}")
        print(f"   Staging Alias: @{self.STAGING_ALIAS}")
        print(f"   Primary Metric: {self.PRIMARY_METRIC}")
        print(f"   Direction: {self.DIRECTION}")

# Initialize config
config = Config()

print("=" * 80)

# =============================================================================
# ✅ INITIALIZE MLFLOW
# =============================================================================
print("\n🔧 Step 2: Initializing MLflow...")

try:
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()
    
    print("✅ MLflow initialized successfully")

except Exception as e:
    print(f"❌ Failed to initialize MLflow: {e}")
    sys.exit(1)

print("\n📢 Staging Promotion Pipeline Started")

# =============================================================================
# 🔧 HELPER FUNCTIONS
# =============================================================================

def wait_until_ready(version: int, timeout: int = 300) -> bool:
    """Wait for model version to become READY"""
    print(f"\n⏳ Waiting for model v{version} to become READY...")

    start = time.time()
    while time.time() - start < timeout:
        mv = client.get_model_version(config.MODEL_NAME, version)
        if mv.status == "READY":
            print(f"   ✅ Model v{version} is READY")
            return True
        elif mv.status == "FAILED_REGISTRATION":
            print(f"   ❌ Model registration failed")
            return False
        time.sleep(5)

    print(f"   ⏰ Timeout waiting for READY status")
    return False


def get_metric_from_run(run_id: str) -> Optional[float]:
    """Fetch primary metric value from MLflow run"""
    try:
        run = client.get_run(run_id)
        metric_value = run.data.metrics.get(config.PRIMARY_METRIC)
        return metric_value
    except Exception as e:
        print(f"   ⚠️ Failed to get metric for run {run_id}: {e}")
        return None


def get_all_registered_versions() -> List[Dict]:
    """Fetch ALL registered model versions with their metrics"""
    print("\n📋 Fetching ALL registered model versions...")
    
    try:
        versions = list(client.search_model_versions(f"name='{config.MODEL_NAME}'"))
        
        if not versions:
            print("❌ No model versions found in registry")
            return []
        
        version_list = []
        for mv in versions:
            metric_value = get_metric_from_run(mv.run_id)
            
            version_info = {
                'version': int(mv.version),
                'run_id': mv.run_id,
                'metric': metric_value,
                'status': mv.status,
                'creation_timestamp': mv.creation_timestamp
            }
            version_list.append(version_info)
            
            print(f"   v{mv.version}: {config.PRIMARY_METRIC}={metric_value} (status={mv.status})")
        
        print(f"\n📊 Total versions found: {len(version_list)}")
        return version_list
        
    except Exception as e:
        print(f"❌ Error fetching model versions: {e}")
        return []


def find_best_model(versions: List[Dict]) -> Optional[Dict]:
    """Find the best model from all versions based on primary metric"""
    print(f"\n🔍 Finding best model based on {config.PRIMARY_METRIC} ({config.DIRECTION})...")
    
    # Filter out versions without metrics
    valid_versions = [v for v in versions if v['metric'] is not None and v['status'] == 'READY']
    
    if not valid_versions:
        print("❌ No valid versions with metrics found")
        return None
    
    # Sort based on direction
    if config.DIRECTION == "maximize":
        best = max(valid_versions, key=lambda x: x['metric'])
        print(f"   📈 Best model (highest {config.PRIMARY_METRIC}): v{best['version']} = {best['metric']:.4f}")
    else:  # minimize
        best = min(valid_versions, key=lambda x: x['metric'])
        print(f"   📉 Best model (lowest {config.PRIMARY_METRIC}): v{best['version']} = {best['metric']:.4f}")
    
    return best


def get_current_staging_version() -> Optional[Dict]:
    """Get the current model in Staging alias"""
    print("\n📋 Checking current Staging model...")
    
    try:
        staging = client.get_model_version_by_alias(config.MODEL_NAME, config.STAGING_ALIAS)
        metric_value = get_metric_from_run(staging.run_id)
        
        staging_info = {
            'version': int(staging.version),
            'run_id': staging.run_id,
            'metric': metric_value
        }
        
        print(f"   Current @{config.STAGING_ALIAS}: v{staging.version} ({config.PRIMARY_METRIC}={metric_value})")
        return staging_info
        
    except Exception as e:
        print(f"   ℹ️ No model currently in @{config.STAGING_ALIAS}")
        return None


def should_promote(best_model: Dict, current_staging: Optional[Dict]) -> tuple:
    """Determine if best model should be promoted to staging"""
    print(f"\n🔬 Comparing best model with current staging...")
    
    # Case 1: No staging model exists - promote best model
    if current_staging is None:
        print("   ✅ No staging model exists - promoting best model")
        return True, "First staging model (best from all versions)"
    
    # Case 2: Best model is already in staging - no change needed
    if best_model['version'] == current_staging['version']:
        print(f"   ℹ️ Best model (v{best_model['version']}) is already in staging")
        return False, "Best model already in staging"
    
    # Case 3: Best model is different from staging - promote it
    print(f"   🔄 Better model found:")
    print(f"      Current staging: v{current_staging['version']} = {current_staging['metric']:.4f}")
    print(f"      Best available: v{best_model['version']} = {best_model['metric']:.4f}")
    
    return True, f"Better model found (v{best_model['version']} > v{current_staging['version']})"


def promote_to_staging(version: int, reason: str) -> bool:
    """Promote a specific version to Staging alias"""
    print(f"\n🚀 Promoting v{version} → @{config.STAGING_ALIAS}")
    print(f"   Reason: {reason}")
    
    # Wait for model to be ready
    if not wait_until_ready(version):
        print("❌ Promotion failed - model not ready")
        return False
    
    try:
        # Set the staging alias
        client.set_registered_model_alias(
            config.MODEL_NAME, 
            config.STAGING_ALIAS, 
            version
        )
        
        print(f"✅ Successfully promoted v{version} to @{config.STAGING_ALIAS}")
        return True
        
    except Exception as e:
        print(f"❌ Failed to promote model: {e}")
        traceback.print_exc()
        return False


# =============================================================================
# 🎬 MAIN EXECUTION
# =============================================================================

def main():
    """Main execution logic - Always promote the best model"""
    
    print("\n" + "=" * 80)
    print("🎯 STARTING BEST MODEL SELECTION AND PROMOTION")
    print("=" * 80)
    
    # Step 1: Get all registered model versions
    all_versions = get_all_registered_versions()
    
    if not all_versions:
        print("\n❌ No model versions found - nothing to promote")
        return
    
    # Step 2: Find the best model from all versions
    best_model = find_best_model(all_versions)
    
    if not best_model:
        print("\n❌ Could not determine best model")
        return
    
    # Step 3: Get current staging model
    current_staging = get_current_staging_version()
    
    # Step 4: Decide if promotion is needed
    should_promote_flag, reason = should_promote(best_model, current_staging)
    
    # Step 5: Promote if needed
    if should_promote_flag:
        success = promote_to_staging(best_model['version'], reason)
        
        if success:
            print("\n" + "=" * 80)
            print("🎉 PROMOTION COMPLETED SUCCESSFULLY!")
            print(f"   Model v{best_model['version']} is now in @{config.STAGING_ALIAS}")
            print(f"   Metric: {config.PRIMARY_METRIC} = {best_model['metric']:.4f}")
            print("=" * 80)
        else:
            print("\n❌ Promotion failed")
    else:
        print("\n" + "=" * 80)
        print("ℹ️ NO PROMOTION NEEDED")
        print(f"   Reason: {reason}")
        print("=" * 80)
    
    print("\n🎉 UAT Staging Promotion process completed!")


if __name__ == "__main__":
    main()